In [22]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from openpyxl.styles.alignment import horizontal_alignments
import dataframe_image as dfi
from PIL import Image, ImageDraw, ImageFont

## Load and preprocess data

In [2]:
# Define converters to handle non-numeric values and convert Year to int
converters = {
    'Total Population, as of 1 July (thousands)': lambda x: pd.to_numeric(x, errors='coerce'),
    'Net Number of Migrants (thousands)': lambda x: pd.to_numeric(x, errors='coerce'),
    'Net Migration Rate (per 1,000 population)': lambda x: pd.to_numeric(x, errors='coerce')
}

# Read the Excel file and specify the converters
df = pd.read_excel(
    '../../data/wpp_2024.xlsx',
    skiprows=16,
    usecols=[
        'Region, subregion, country or area *',
        'Type',
        'Year',
        'Total Population, as of 1 July (thousands)',
        'Net Number of Migrants (thousands)',
        'Net Migration Rate (per 1,000 population)'
    ],
    converters=converters
)

In [3]:
df = df.rename(
    columns={'Region, subregion, country or area *': 'Country',
             'Total Population, as of 1 July (thousands)': 'Population',
            'Net Number of Migrants (thousands)': 'Net Migrants (thousands)',}
)

In [4]:
df.dtypes

Country                                       object
Type                                          object
Year                                         float64
Population                                   float64
Net Migrants (thousands)                     float64
Net Migration Rate (per 1,000 population)    float64
dtype: object

In [5]:
# Add Net migration rate as percentage of population
df['Net Migration Rate (%)'] = ((df['Net Migrants (thousands)'] / df['Population']) * 100).round(2)

In [6]:
df.columns

Index(['Country', 'Type', 'Year', 'Population', 'Net Migrants (thousands)',
       'Net Migration Rate (per 1,000 population)', 'Net Migration Rate (%)'],
      dtype='object')

## Rank countries by Negative Net Migration Rate (per 1,000 population) in 2023

In [7]:

df_2023_ranked = df.query('Year == 2023').sort_values(
    'Net Migration Rate (per 1,000 population)',ascending=True)

df_2023_ranked

,Country,Type,Year,Population,Net Migrants (thousands),"Net Migration Rate (per 1,000 population)",Net Migration Rate (%)
17320,Saint Martin (French part),Country/Area,2023.0,27.515,-1.697,-61.677,-6.17
20946,Marshall Islands,Country/Area,2023.0,38.827,-1.832,-47.184,-4.72
21464,Cook Islands,Country/Area,2023.0,14.222,-0.558,-39.235,-3.92
21908,Tuvalu,Country/Area,2023.0,9.816,-0.323,-32.907,-3.29
21168,Northern Mariana Islands,Country/Area,2023.0,45.143,-1.321,-29.263,-2.93
...,...,...,...,...,...,...,...
11325,United Arab Emirates,Country/Area,2023.0,10642.081,300.004,28.190,2.82
21760,Tokelau,Country/Area,2023.0,2.397,0.073,30.448,3.05
10881,Oman,Country/Area,2023.0,5049.269,156.285,30.952,3.10
11177,Syrian Arab Republic,Country/Area,2023.0,23594.623,757.309,32.097,3.21


In [8]:
# List of war-torn countries or in war-like conditions in 2023
to_remove_war_torn = [
    "Afghanistan", "Syrian Arab Republic", "Iraq", "Yemen", "Libya", "Somalia", "South Sudan", "Armenia", "Azerbaijan", "Kosovo (under UNSC res. 1244)", "Ukraine", "Sudan", "Russian Federation", "Lebanon", "State of Palestine", 
]

In [9]:
df_2023_ranked = df_2023_ranked[~df_2023_ranked['Country'].isin(to_remove_war_torn)]
df_2023_ranked

,Country,Type,Year,Population,Net Migrants (thousands),"Net Migration Rate (per 1,000 population)",Net Migration Rate (%)
17320,Saint Martin (French part),Country/Area,2023.0,27.515,-1.697,-61.677,-6.17
20946,Marshall Islands,Country/Area,2023.0,38.827,-1.832,-47.184,-4.72
21464,Cook Islands,Country/Area,2023.0,14.222,-0.558,-39.235,-3.92
21908,Tuvalu,Country/Area,2023.0,9.816,-0.323,-32.907,-3.29
21168,Northern Mariana Islands,Country/Area,2023.0,45.143,-1.321,-29.263,-2.93
...,...,...,...,...,...,...,...
4443,Chad,Country/Area,2023.0,19319.064,496.830,25.717,2.57
16062,"Bonaire, Sint Eustatius and Saba",Country/Area,2023.0,29.898,0.820,27.426,2.74
11325,United Arab Emirates,Country/Area,2023.0,10642.081,300.004,28.190,2.82
21760,Tokelau,Country/Area,2023.0,2.397,0.073,30.448,3.05


In [10]:
# Keep only Country/Area type with population greater than 50 thousand
df_2023_ranked = df_2023_ranked.query('Type == "Country/Area" and Population > 50').reset_index(drop=True)
df_2023_ranked

,Country,Type,Year,Population,Net Migrants (thousands),"Net Migration Rate (per 1,000 population)",Net Migration Rate (%)
0,Tonga,Country/Area,2023.0,104.597,-2.173,-20.775,-2.08
1,Greece,Country/Area,2023.0,10242.908,-159.430,-15.565,-1.56
2,Nepal,Country/Area,2023.0,29694.614,-409.782,-13.800,-1.38
3,Samoa,Country/Area,2023.0,216.663,-2.810,-12.969,-1.30
4,Micronesia (Fed. States of),Country/Area,2023.0,112.630,-1.134,-10.069,-1.01
...,...,...,...,...,...,...,...
190,Lithuania,Country/Area,2023.0,2854.099,44.935,15.744,1.57
191,Montenegro,Country/Area,2023.0,633.552,12.000,18.941,1.89
192,Chad,Country/Area,2023.0,19319.064,496.830,25.717,2.57
193,United Arab Emirates,Country/Area,2023.0,10642.081,300.004,28.190,2.82


In [11]:
# Country/Area with population greater than 125 thousand
top_20 = df_2023_ranked.query(
    'Type == "Country/Area" and Population > 125'
    ).drop(
        columns=['Type', 'Year']
    ).reset_index(drop=True).head(20)

top_20.index = top_20.index + 1
# top_20['rank'] = top_20.index

In [18]:
styled_top_20 = top_20.style.set_properties(**{'text-align': 'center'}).set_table_styles(
    [{'selector': 'th', 'props': [('text-align', 'center')]}]
)

In [19]:
styled_top_20

,Country,Population,Net Migrants (thousands),"Net Migration Rate (per 1,000 population)",Net Migration Rate (%)
1,Greece,10242.908000,-159.430000,-15.565000,-1.560000
2,Nepal,29694.614000,-409.782000,-13.800000,-1.380000
3,Samoa,216.663000,-2.810000,-12.969000,-1.300000
4,Albania,2811.655000,-25.357000,-9.019000,-0.900000
5,Martinique,346.002000,-3.078000,-8.896000,-0.890000
6,Guyana,826.353000,-6.161000,-7.456000,-0.750000
7,Eswatini,1230.506000,-8.549000,-6.948000,-0.690000
8,Pakistan,247504.495000,-1619.557000,-6.544000,-0.650000
9,Republic of Moldova,3067.070000,-18.570000,-6.055000,-0.610000
10,Zimbabwe,16340.822000,-97.380000,-5.959000,-0.600000


In [26]:
dfi.export(styled_top_20, 'top20_2023.png', table_conversion="selenium")

In [39]:
# Open the image and add the title
image = Image.open('top20_2023.png')
draw = ImageDraw.Draw(image)
title = "Top 20 countries in Negative Migration as a percentage of the population, 2023"

# Define font and size (you may need to adjust the path to the font file)
font = ImageFont.truetype("arial.ttf", 24)

# y = 10  # Adjust the y position as needed
# Create a new image with space for the title
# Calculate the width and height of the title to center it
text_bbox = draw.textbbox((0, 0), title, font=font)
text_width = text_bbox[2] - text_bbox[0]
text_height = text_bbox[3] - text_bbox[1]
image_width, image_height = image.size
x = (image_width - text_width) / 2

# Create a new image with space for the title
new_image_height = image_height + text_height + 20  # Add some padding
new_image = Image.new('RGB', (image_width, new_image_height), 'white')
new_draw = ImageDraw.Draw(new_image)

# Add the title to the new image
new_draw.text((x, 10), title, font=font, fill="black")

# Paste the original image below the title
new_image.paste(image, (0, text_height + 20))


# Save the image with the title
new_image.save('top20_2023_with_title.png')

In [54]:
# Open the image and add the title and subtitle
image = Image.open('top20_2023.png')
draw = ImageDraw.Draw(image)
title = "Top 20 countries in Negative Net Migration as a percentage of the population, 2023"
subtitle = "source: United Nations Population Prospects 2024, excluding war-torn areas and small nation states."

# Define font and size (you may need to adjust the path to the font file)
font_title = ImageFont.truetype("arial.ttf", 24)
font_subtitle = ImageFont.truetype("arial.ttf", 14)

# Calculate the width and height of the title and subtitle to center them
title_bbox = draw.textbbox((0, 0), title, font=font_title)
title_width = title_bbox[2] - title_bbox[0]
title_height = title_bbox[3] - title_bbox[1]

subtitle_bbox = draw.textbbox((0, 0), subtitle, font=font_subtitle)
subtitle_width = subtitle_bbox[2] - subtitle_bbox[0]
subtitle_height = subtitle_bbox[3] - subtitle_bbox[1]

image_width, image_height = image.size
x_title = (image_width - title_width) / 2
x_subtitle = 10  # (image_width - subtitle_width) / 2  # Aling left instead of center

# Create a new image with space for the title, table, and subtitle
new_image_height = image_height + title_height + subtitle_height + 45  # Add some padding
new_image = Image.new('RGB', (image_width, new_image_height), 'white')
new_draw = ImageDraw.Draw(new_image)

# Add the title to the new image
new_draw.text((x_title, 10), title, font=font_title, fill="black")

# Paste the original image below the title
new_image.paste(image, (0, title_height + 20))

# Add the subtitle below the table
new_draw.text((x_subtitle, title_height + image_height + 35), subtitle, font=font_subtitle, fill="black")

# Save the new image with the title, table, and subtitle
new_image.save('top20_2023_with_title_and_subtitle.png')

In [ ]:

# to_be_first_col = 'rank'

# col = top_20.pop(to_be_first_col)


# top_20.insert(0, 'rank', col)

In [ ]:
top_20


In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Assuming top_20 is your DataFrame
table_top_20 = go.Figure(data=[go.Table(
    header=dict(values=list(top_20.columns),
                #=[f"{col}" for col in top_20.columns],  # Add line breaks for vertical centering
                fill_color='black',  # Set header background color to black
                align='left',  # Align header text to the center horizontally
                font=dict(color='white')),  # Set header font color to white
    cells=dict(values=[top_20[col] for col in top_20.columns],
               fill_color='black',  # Set cells background color to black
               align='left',  # Align cells text to the left
               font=dict(color='white'))  # Set cells font color to white
)])

# Save the interactive table as a PNG file
table_top_20.write_image('top_20_table.png')

# Display the interactive table in a Jupyter Notebook (optional)
table_top_20.show()

In [ ]:
# Filter the DataFrame for rows where the area is Greece and the Year is between 1989 and 2023
df_greece_filtered = df.query('Country == "Greece" and 1950 <= Year <= 2023')

# Select the specified columns
df_greece_table = df_greece_filtered[
    ['Country', 'Year', 'Net Migrants (thousands)', 'Net Migration Rate (per 1,000 population)']]

# Display the table
df_greece_table

In [ ]:
df_greece_table.tail(36)

In [ ]:
# Sum the Net Number of Migrants (thousands) for greece from 1950 to 2023
df_greece_filtered['Net Number of Migrants (thousands)'].sum()

In [ ]:
# sum net number of migrants of Greece from 1990 to 2023    
df_greece_filtered.query('Year >= 1990')['Net Number of Migrants (thousands)'].sum()

In [ ]:
# sum net number of migrants of Greece from 1990 to 2023 

In [ ]:
import pandas as pd
import plotly.express as px

# Filter the DataFrame for rows where the area is Greece and the Year is between 1950 and 2023
df_greece_filtered = df.query('area == "Greece" and 1950 <= Year <= 2023')

# Create the first chart for 'Net Number of Migrants (thousands)'
fig1 = px.line(df_greece_filtered, x='Year', y='Net Number of Migrants (thousands)',
               title='Net Number of Migrants (thousands) in Greece (1950-2023)',
               labels={'Net Number of Migrants (thousands)': 'Net Number of Migrants (thousands)'})

# Show the first chart
fig1.show()

# Create the second chart for 'Net Migration Rate'
fig2 = px.line(df_greece_filtered, x='Year', y='Net Migration Rate (per 1,000 population)',
               title='Net Migration Rate (per 1,000 population) in Greece (1950-2023)',
               labels={'Net Migration Rate (per 1,000 population)': 'Net Migration Rate (per 1,000 population)'})

# Show the second chart
fig2.show()

In [ ]:
import pandas as pd
import plotly.express as px

# Filter the DataFrame for rows where the area is Greece and the Year is between 1950 and 2023
df_greece_filtered = df.query('area == "Greece" and 1950 <= Year <= 2023')

# Create the first chart for 'Net Number of Migrants (thousands)'
fig1 = px.line(df_greece_filtered, x='Year', y='Net Number of Migrants (thousands)',
               title='Net Number of Migrants (thousands) in Greece (1950-2023)',
               labels={'Net Number of Migrants (thousands)': 'Net Number of Migrants (thousands)'})

# Add a horizontal line at y=0
fig1.add_shape(
    type='line',
    x0=df_greece_filtered['Year'].min(),
    x1=df_greece_filtered['Year'].max(),
    y0=0,
    y1=0,
    line=dict(
        color='White',
        width=2  # Three times thicker than the default grid lines
    )
)

# Show the first chart
fig1.show()

# Create the second chart for 'Net Migration Rate'
fig2 = px.line(df_greece_filtered, x='Year', y='Net Migration Rate (per 1,000 population)',
               title='Net Migration Rate (per 1,000 population) in Greece (1950-2023)',
               labels={'Net Migration Rate (per 1,000 population)': 'Net Migration Rate (per 1,000 population)'})

# Add a horizontal line at y=0
fig2.add_shape(
    type='line',
    x0=df_greece_filtered['Year'].min(),
    x1=df_greece_filtered['Year'].max(),
    y0=0,
    y1=0,
    line=dict(
        color='White',
        width=2  # Three times thicker than the default grid lines
    )
)

# Show the second chart
fig2.show()

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Filter the DataFrame for rows where the area is Greece and the Year is between 1950 and 2023
df_greece_filtered = df.query('area == "Greece" and 1950 <= Year <= 2023')

# Create the first chart for 'Net Number of Migrants (thousands)'
fig1 = go.Figure()

# Add points above zero
fig1.add_trace(go.Scatter(
    x=df_greece_filtered['Year'],
    y=df_greece_filtered['Net Number of Migrants (thousands)'],
    mode='lines+markers',
    marker=dict(color=['green' if y > 0 else 'red' for y in df_greece_filtered['Net Number of Migrants (thousands)']],
    line=dict(color='#283442'))
))




# Add a horizontal line at y=0
fig1.add_shape(
    type='line',
    x0=df_greece_filtered['Year'].min()-5,
    x1=df_greece_filtered['Year'].max()+5,
    y0=0,
    y1=0,
    line=dict(
        color='White',
        width=1
    )
)

# Update layout
fig1.update_layout(
    title='Net Number of Migrants (thousands) in Greece (1950-2023)',
    xaxis_title='Year',
    yaxis_title='Net Number of Migrants (thousands)',
    xaxis=dict(showgrid=False),  # Remove x-axis grid lines
    yaxis=dict(showgrid=False)   # Remove y-axis grid lines
)

# Show the first chart
fig1.show()

In [ ]:
##########################
# Create the second chart for 'Net Migration Rate'
fig2 = go.Figure()

# Add points above zero
fig2.add_trace(go.Scatter(
    x=df_greece_filtered['Year'],
    y=df_greece_filtered['Net Migration Rate (per 1,000 population)'],
    mode='lines+markers',
    marker=dict(color=['green' if y > 0 else 'red' for y in df_greece_filtered['Net Migration Rate (per 1,000 population)']]),
    line=dict(color='#506784',
              width=2)
))

# Add a horizontal line at y=0
fig2.add_shape(
    type='line',
    x0=df_greece_filtered['Year'].min(),
    x1=df_greece_filtered['Year'].max(),
    y0=0,
    y1=0,
    line=dict(
        color='White',
        width=2
    )
)

# Update layout
fig2.update_layout(
    title='Net Migration Rate (per 1,000 population) in Greece (1950-2023)',
    xaxis_title='Year',
    yaxis_title='Net Migration Rate (per 1,000 population)'
)

# Show the second chart
fig2.show()

In [ ]:

# Extract grid colors from the figure
xaxis_grid_color = fig2.layout.xaxis.gridcolor
yaxis_grid_color = fig2.layout.yaxis.gridcolor

print(f"X-axis grid color: {xaxis_grid_color}")
print(f"Y-axis grid color: {yaxis_grid_color}")

In [ ]:
# Extract theme colors from the figure
theme_colors = {
    'background_color': fig2.layout.template.layout.paper_bgcolor,
    'plot_background_color': fig2.layout.template.layout.plot_bgcolor,
    'grid_color': fig2.layout.template.layout.xaxis.gridcolor,
    'font_color': fig2.layout.template.layout.font.color,
    'axis_line_color': fig2.layout.template.layout.xaxis.linecolor
}

print(theme_colors)

In [ ]:

# Extract grid colors from the figure
xaxis_grid_color = fig2.layout.xaxis.gridcolor
yaxis_grid_color = fig2.layout.yaxis.gridcolor

print(f"X-axis grid color: {xaxis_grid_color}")
print(f"Y-axis grid color: {yaxis_grid_color}")

In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Filter the DataFrame for rows where the area is Greece and the Year is between 1950 and 2023
df_greece_filtered = df.query('area == "Greece" and 1950 <= Year <= 2023')

# Create the first chart for 'Net Number of Migrants (thousands)'
fig1 = go.Figure()

# Add points above zero
fig1.add_trace(go.Scatter(
    x=df_greece_filtered['Year'],
    y=df_greece_filtered['Net Number of Migrants (thousands)'],
    mode='lines+markers',
    marker=dict(color=['green' if y > 0 else 'red' for y in df_greece_filtered['Net Number of Migrants (thousands)']]),
    line=dict(color='lightgrey')  # Set line color to match grid color
))

# Add a horizontal line at y=0
fig1.add_shape(
    type='line',
    x0=df_greece_filtered['Year'].min(),
    x1=df_greece_filtered['Year'].max(),
    y0=0,
    y1=0,
    line=dict(
        color='Black',
        width=3  # Three times thicker than the default grid lines
    )
)

# Update layout
fig1.update_layout(
    title='Net Number of Migrants (thousands) in Greece (1950-2023)',
    xaxis_title='Year',
    yaxis_title='Net Number of Migrants (thousands)'
)

# Show the first chart
fig1.show()

# Create the second chart for 'Net Migration Rate'
fig2 = go.Figure()

# Add points above zero
fig2.add_trace(go.Scatter(
    x=df_greece_filtered['Year'],
    y=df_greece_filtered['Net Migration Rate (per 1,000 population)'],
    mode='lines+markers',
    marker=dict(color=['green' if y > 0 else 'red' for y in df_greece_filtered['Net Migration Rate (per 1,000 population)']]),
    line=dict(color='lightgrey')  # Set line color to match grid color
))

# Add a horizontal line at y=0
fig2.add_shape(
    type='line',
    x0=df_greece_filtered['Year'].min(),
    x1=df_greece_filtered['Year'].max(),
    y0=0,
    y1=0,
    line=dict(
        color='Black',
        width=3  # Three times thicker than the default grid lines
    )
)

# Update layout
fig2.update_layout(
    title='Net Migration Rate (per 1,000 population) in Greece (1950-2023)',
    xaxis_title='Year',
    yaxis_title='Net Migration Rate (per 1,000 population)'
)

# Show the second chart
fig2.show()

In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Filter the DataFrame for rows where the area is Greece and the Year is between 1950 and 2023
df_greece_filtered = df.query('area == "Greece" and 1950 <= Year <= 2023')

# Create the first chart for 'Net Number of Migrants (thousands)'
fig1 = go.Figure()

# Add points above zero
fig1.add_trace(go.Scatter(
    x=df_greece_filtered['Year'],
    y=df_greece_filtered['Net Number of Migrants (thousands)'],
    mode='lines+markers',
    marker=dict(color=['green' if y > 0 else 'red' for y in df_greece_filtered['Net Number of Migrants (thousands)']]),
    line=dict(color='#e5ecf6')  # Set line color to match grid color
))

# Add a horizontal line at y=0
fig1.add_shape(
    type='line',
    x0=df_greece_filtered['Year'].min(),
    x1=df_greece_filtered['Year'].max(),
    y0=0,
    y1=0,
    line=dict(
        color='white',  # Set horizontal line color to white
        width=3  # Three times thicker than the default grid lines
    )
)

# Update layout
fig1.update_layout(
    title='Net Number of Migrants (thousands) in Greece (1950-2023)',
    xaxis_title='Year',
    yaxis_title='Net Number of Migrants (thousands)',
    xaxis=dict(showgrid=False),  # Remove x-axis grid lines
    yaxis=dict(showgrid=False)   # Remove y-axis grid lines
)

# Show the first chart
fig1.show()

# Create the second chart for 'Net Migration Rate'
fig2 = go.Figure()

# Add points above zero
fig2.add_trace(go.Scatter(
    x=df_greece_filtered['Year'],
    y=df_greece_filtered['Net Migration Rate (per 1,000 population)'],
    mode='lines+markers',
    marker=dict(color=['green' if y > 0 else 'red' for y in df_greece_filtered['Net Migration Rate (per 1,000 population)']]),
    line=dict(color='#e5ecf6')  # Set line color to match grid color
))

# Add a horizontal line at y=0
fig2.add_shape(
    type='line',
    x0=df_greece_filtered['Year'].min(),
    x1=df_greece_filtered['Year'].max(),
    y0=0,
    y1=0,
    line=dict(
        color='white',  # Set horizontal line color to white
        width=3  # Three times thicker than the default grid lines
    )
)

# Update layout
fig2.update_layout(
    title='Net Migration Rate (per 1,000 population) in Greece (1950-2023)',
    xaxis_title='Year',
    yaxis_title='Net Migration Rate (per 1,000 population)',
    xaxis=dict(showgrid=False),  # Remove x-axis grid lines
    yaxis=dict(showgrid=False)   # Remove y-axis grid lines
)

# Show the second chart
fig2.show()

In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Filter the DataFrame for rows where the area is Greece and the Year is between 1950 and 2023
df_greece_filtered = df.query('area == "Greece" and 1950 <= Year <= 2023')

def create_segments(df, y_column):
    segments = []
    current_segment = {'x': [], 'y': [], 'color': 'green' if df[y_column].iloc[0] > 0 else 'red'}
    
    for i in range(len(df)):
        current_y = df[y_column].iloc[i]
        current_segment['x'].append(df['Year'].iloc[i])
        current_segment['y'].append(current_y)
        
        if i < len(df) - 1:
            next_y = df[y_column].iloc[i + 1]
            if (current_y > 0 and next_y <= 0) or (current_y <= 0 and next_y > 0):
                segments.append(current_segment)
                current_segment = {'x': [df['Year'].iloc[i]], 'y': [current_y], 'color': 'green' if next_y > 0 else 'red'}
    
    segments.append(current_segment)
    return segments

# Create the first chart for 'Net Number of Migrants (thousands)'
fig1 = go.Figure()

segments = create_segments(df_greece_filtered, 'Net Number of Migrants (thousands)')
for segment in segments:
    fig1.add_trace(go.Scatter(
        x=segment['x'],
        y=segment['y'],
        mode='lines',
        line=dict(color=segment['color'])
    ))

# Add a horizontal line at y=0
fig1.add_shape(
    type='line',
    x0=df_greece_filtered['Year'].min(),
    x1=df_greece_filtered['Year'].max(),
    y0=0,
    y1=0,
    line=dict(
        color='Black',
        width=3  # Three times thicker than the default grid lines
    )
)

# Update layout
fig1.update_layout(
    title='Net Number of Migrants (thousands) in Greece (1950-2023)',
    xaxis_title='Year',
    yaxis_title='Net Number of Migrants (thousands)'
)

# Show the first chart
fig1.show()

# Create the second chart for 'Net Migration Rate'
fig2 = go.Figure()

segments = create_segments(df_greece_filtered, 'Net Migration Rate (per 1,000 population)')
for segment in segments:
    fig2.add_trace(go.Scatter(
        x=segment['x'],
        y=segment['y'],
        mode='lines',
        line=dict(color=segment['color'])
    ))

# Add a horizontal line at y=0
fig2.add_shape(
    type='line',
    x0=df_greece_filtered['Year'].min(),
    x1=df_greece_filtered['Year'].max(),
    y0=0,
    y1=0,
    line=dict(
        color='Black',
        width=3  # Three times thicker than the default grid lines
    )
)

# Update layout
fig2.update_layout(
    title='Net Migration Rate (per 1,000 population) in Greece (1950-2023)',
    xaxis_title='Year',
    yaxis_title='Net Migration Rate (per 1,000 population)'
)

# Show the second chart
fig2.show()

In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Filter the DataFrame for rows where the area is Greece and the Year is between 1950 and 2023
df_greece_filtered = df.query('area == "Greece" and 1950 <= Year <= 2023')

def create_segments(df, y_column):
    segments = []
    current_segment = {'x': [], 'y': [], 'color': 'green' if df[y_column].iloc[0] > 0 else 'red'}

    for i in range(len(df)):
        current_y = df[y_column].iloc[i]
        current_segment['x'].append(df['Year'].iloc[i])
        current_segment['y'].append(current_y)

        if i < len(df) - 1:
            next_y = df[y_column].iloc[i + 1]
            if (current_y > 0 and next_y <= 0) or (current_y <= 0 and next_y > 0):
                segments.append(current_segment)
                current_segment = {'x': [df['Year'].iloc[i]], 'y': [current_y], 'color': 'green' if next_y > 0 else 'red'}

    segments.append(current_segment)
    return segments

# Create the first chart for 'Net Number of Migrants (thousands)'
fig1 = go.Figure()

segments = create_segments(df_greece_filtered, 'Net Number of Migrants (thousands)')
for segment in segments:
    fig1.add_trace(go.Scatter(
        x=segment['x'],
        y=segment['y'],
        mode='lines',
        line=dict(color='lightgrey')  # Set line color to match grid color
    ))

# Add a horizontal line at y=0
fig1.add_shape(
    type='line',
    x0=df_greece_filtered['Year'].min(),
    x1=df_greece_filtered['Year'].max(),
    y0=0,
    y1=0,
    line=dict(
        color='white',  # Set horizontal line color to white
        width=3  # Three times thicker than the default grid lines
    )
)

# Update layout to remove grid lines
fig1.update_layout(
    title='Net Number of Migrants (thousands) in Greece (1950-2023)',
    xaxis_title='Year',
    yaxis_title='Net Number of Migrants (thousands)',
    xaxis=dict(showgrid=False),  # Remove x-axis grid lines
    yaxis=dict(showgrid=False)   # Remove y-axis grid lines
)

# Show the first chart
fig1.show()

# Create the second chart for 'Net Migration Rate'
fig2 = go.Figure()

segments = create_segments(df_greece_filtered, 'Net Migration Rate (per 1,000 population)')
for segment in segments:
    fig2.add_trace(go.Scatter(
        x=segment['x'],
        y=segment['y'],
        mode='lines',
        line=dict(color='lightgrey')  # Set line color to match grid color
    ))

# Add a horizontal line at y=0
fig2.add_shape(
    type='line',
    x0=df_greece_filtered['Year'].min(),
    x1=df_greece_filtered['Year'].max(),
    y0=0,
    y1=0,
    line=dict(
        color='white',  # Set horizontal line color to white
        width=3  # Three times thicker than the default grid lines
    )
)

# Update layout to remove grid lines
fig2.update_layout(
    title='Net Migration Rate (per 1,000 population) in Greece (1950-2023)',
    xaxis_title='Year',
    yaxis_title='Net Migration Rate (per 1,000 population)',
    xaxis=dict(showgrid=False),  # Remove x-axis grid lines
    yaxis=dict(showgrid=False)   # Remove y-axis grid lines
)

# Show the second chart
fig2.show()

In [ ]:
# Filter the DataFrame for rows where the "Type" is "Country/Area" and "Total Population, as of 1 July (thousands)" is >= 100
df_filtered = df_2023_selected.query('Type == "Country/Area" and `Total Population, as of 1 July (thousands)` >= 100')

# Remove specified countries
countries_to_remove = [
    "Kosovo (under UNSC res. 1244)", "Ukraine", "Sudan", "Azerbaijan", "Lebanon", "State of Palestine"]
df_filtered = df_filtered[~df_filtered['area'].isin(countries_to_remove)]

# Sort the filtered DataFrame by 'Net Migration Rate (per 1,000 population)' in ascending order and reset the index
df_sorted = df_filtered.sort_values('Net Migration Rate (per 1,000 population)', ascending=True).reset_index(drop=True)

# Drop the specified columns from the DataFrame
df_sorted = df_sorted.drop(columns=['ISO3 Alpha-code', 'Type', 'Year'])

# Display the updated DataFrame
df_sorted

In [ ]:

# Sort the DataFrame and select the top 10 countries
top_10_countries = df_2023_selected.sort_values(('Net Migration Rate (per 1,000 population)'), ascending=True).head(10)

# Create a choropleth map
fig = px.choropleth(
    top_10_countries,
    locations='ISO3 Alpha-code',
    color='Net Migration Rate (per 1,000 population)',
    hover_name='area',
    color_continuous_scale=px.colors.sequential.Plasma,
    title='Top 10 Countries by Net Migration Rate (per 1,000 population)'
)

# Show the map
fig.show()

In [ ]:

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Sort the DataFrame and select the top 10 countries
top_10_countries = df_2023_selected.sort_values(('Net Migration Rate (per 1,000 population)'), ascending=True).head(10)

# Create a choropleth map
fig = px.choropleth(
    top_10_countries,
    locations='ISO3 Alpha-code',
    color='Net Migration Rate (per 1,000 population)',
    hover_name='area',
    color_continuous_scale=px.colors.sequential.Plasma,
    title='Top 10 Countries by Net Migration Rate (per 1,000 population)'
)

# Extract the color scale
color_scale = px.colors.sequential.Plasma

# Create custom legend items
custom_legend = [
    go.Scatter(
        x=[None],
        y=[None],
        mode='markers',
        marker=dict(
            size=10,
            color=color_scale[i],
        ),
        legendgroup=f"{row.area} ({row._8})",
        showlegend=True,
        name=f"{row.area} ({row._8})"
    ) for i, row in enumerate(top_10_countries.itertuples())
]

# Add custom legend items to the figure
for item in custom_legend:
    fig.add_trace(item)

# Update the layout to position the legend outside the plot
fig.update_layout(
    legend=dict(
        x=1.05,
        y=1,
        traceorder='normal',
        font=dict(
            family='sans-serif',
            size=12,
            color='black'
        ),
        bgcolor='rgba(0,0,0,0)',
        bordercolor='rgba(0,0,0,0)'
    )
)

# Show the map
fig.show()

In [ ]:

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Sort the DataFrame and select the top 10 countries
top_10_countries = df_2023_selected.sort_values(('Net Migration Rate (per 1,000 population)'), ascending=True).head(10)

# Create a choropleth map
fig = px.choropleth(
    top_10_countries,
    locations='ISO3 Alpha-code',
    color='Net Migration Rate (per 1,000 population)',
    hover_name='area',
    color_continuous_scale=px.colors.sequential.Plasma,
    title='Top 10 Countries by Net Migration Rate (per 1,000 population)'
)

# Extract the color scale
color_scale = px.colors.sequential.Plasma

# Create custom legend items
custom_legend = [
    go.Scatter(
        x=[None],
        y=[None],
        mode='markers',
        marker=dict(
            size=10,
            color=color_scale[i],
        ),
        showlegend=True,
        name=f"{row._8} ({row.area})"
    ) for i, row in enumerate(top_10_countries.itertuples())
]

# Add custom legend items to the figure
for item in custom_legend:
    fig.add_trace(item)

# Update the layout to position the legend outside the plot
fig.update_layout(
    legend=dict(
        x=1.05,
        y=1,
        traceorder='normal',
        font=dict(
            family='sans-serif',
            size=12,
            color='black'
        ),
        bgcolor='rgba(0,0,0,0)',
        bordercolor='rgba(0,0,0,0)'
    )
)

# Show the map
fig.show()

In [ ]:
df_2023_selected.sort_values(('Net Migration Rate (per 1,000 population)'), ascending=False).query('area == "Greece"')

In [ ]:
df.query('area == "Greece" and Year == 2015')

In [ ]:
df_greece = df_2023_selected.query('area == "Greece"')

In [ ]:
df_greece

In [ ]:
(df_greece["Net Number of Migrants (thousands)"]/df_greece["Total Population, as of 1 July (thousands)"]) * 1000

In [ ]:
(df_greece["Net Number of Migrants (thousands)"]/df_greece["Total Population, as of 1 July (thousands)"]) * 100